In [1]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv("/home/sebastian/Downloads/Temperaturas.zip",index_col=0)
df.head()

,sensor,fecha,temperatura
id,,,
2,sala,2020-03-31 14:38:14,23.80
3,oficina,2020-03-31 14:43:40,27.20
4,piscina,2020-03-31 14:46:26,27.62
5,sala,2020-03-31 14:48:14,25.40
6,oficina,2020-03-31 14:53:40,28.80


In [17]:
ndf=df[df["sensor"]=='piscina']
ndf.head()

,sensor,fecha,temperatura
id,,,
4,piscina,2020-03-31 14:46:26,27.62
7,piscina,2020-03-31 14:56:32,27.56
10,piscina,2020-03-31 15:06:37,27.62
13,piscina,2020-03-31 15:16:42,27.62
16,piscina,2020-03-31 15:26:47,27.62
